In [0]:
dbutils.widgets.text("catalog_name", "renjiharold_demo", "Catalog Name")
dbutils.widgets.text("schema_name", "waf_db", "Schema Name")

In [0]:
catalog = dbutils.widgets.get("catalog_name")
schema = dbutils.widgets.get("schema_name")

if not catalog.strip():
    raise ValueError("Catalog Name widget input cannot be empty.")
if not schema.strip():
    raise ValueError("Schema Name widget input cannot be empty.")

In [0]:
notebooks = [
    "datasets/waf_controls_reliability",
    "datasets/waf_controls_performance"
]

for nb in notebooks:
    try:
        dbutils.notebook.run(nb, 0, {"catalog": catalog, "schema": schema})
        print(f"Successful execution of notebook: {nb}")
    except Exception as e:
        print(f"Failed execution of notebook: {nb} — {e}")

In [0]:
%sql
SELECT *
FROM
  (
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY pillar ORDER BY run_date DESC) as rn
    FROM IDENTIFIER(:catalog_name || '.' || :schema_name || '.waf_log')
  ) log
WHERE
  rn = 1